In [15]:
"""
Análisis COMPLETO del Universo - Tickers ACTIVOS + INACTIVOS
Notebook: 01_DayBook/fase_01/A_Universo/analysis.ipynb
"""
import polars as pl
from pathlib import Path
import os

# =============================================================================
# CONFIGURAR DIRECTORIO DE TRABAJO
# =============================================================================
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)
print(f"📂 Directorio de trabajo: {os.getcwd()}")
print()

# =============================================================================
# CARGAR DATOS - SNAPSHOT ACTIVOS
# =============================================================================
snapshot_path = project_root / "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-19/tickers.parquet"
print(f"📥 Cargando datos desde: {snapshot_path}")
print(f"   Archivo existe: {snapshot_path.exists()}")
print()

df = pl.read_parquet(snapshot_path)

print("=" * 100)
print("ANÁLISIS UNIVERSO COMPLETO - TICKERS ACTIVOS + INACTIVOS")
print("=" * 100)
print()

# =============================================================================
# INFORMACIÓN GENERAL
# =============================================================================
print(f"TOTAL TICKERS (todos): {len(df):,}")
print(f"TOTAL COLUMNAS: {len(df.columns)}")
print()

# Separar activos e inactivos
df_active = df.filter(pl.col("active") == True)
df_inactive = df.filter(pl.col("active") == False)

print(f"✅ TICKERS ACTIVOS:   {len(df_active):>6,} ({len(df_active)/len(df)*100:>5.1f}%)")
print(f"❌ TICKERS INACTIVOS: {len(df_inactive):>6,} ({len(df_inactive)/len(df)*100:>5.1f}%)")
print()

# =============================================================================
# COLUMNAS Y TIPOS DE DATOS
# =============================================================================
print("=" * 100)
print("COLUMNAS Y TIPOS DE DATOS")
print("=" * 100)
for col in df.columns:
    dtype = df[col].dtype
    null_count = df[col].null_count()
    null_pct = (null_count / len(df)) * 100
    print(f"{col:30s} | {str(dtype):20s} | Nulls: {null_count:6,} ({null_pct:5.1f}%)")
print()

# =============================================================================
# HEAD(10) - PRIMERAS 10 ACCIONES (TODOS)
# =============================================================================
print("=" * 100)
print("HEAD(10) - PRIMERAS 10 ACCIONES (ACTIVOS + INACTIVOS)")
print("=" * 100)
cols_display = ["ticker", "name", "market", "primary_exchange", "type", "active", "cik"]
display(df.select(cols_display).head(10))
print()

# =============================================================================
# HEAD(10) - INACTIVOS
# =============================================================================
if len(df_inactive) > 0:
    print("=" * 100)
    print(f"HEAD(10) - PRIMERAS 10 ACCIONES INACTIVAS (Total: {len(df_inactive):,})")
    print("=" * 100)
    display(df_inactive.select(cols_display).head(10))
    print()

# =============================================================================
# TAIL(10) - ÚLTIMAS 10 ACCIONES
# =============================================================================
print("=" * 100)
print("TAIL(10) - ÚLTIMAS 10 ACCIONES")
print("=" * 100)
display(df.select(cols_display).tail(10))
print()

# =============================================================================
# COMPARACIÓN: ACTIVOS VS INACTIVOS
# =============================================================================
print("=" * 100)
print("COMPARACIÓN: ACTIVOS VS INACTIVOS")
print("=" * 100)

# Por tipo de activo
print("\n DISTRIBUCIÓN POR TIPO DE ACTIVO (type):")
print("\n   ACTIVOS:")
type_dist_active = (
    df_active.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_dist_active)

if len(df_inactive) > 0:
    print("\n   INACTIVOS:")
    type_dist_inactive = (
        df_inactive.group_by("type")
        .agg(pl.len().alias("count"))
        .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
        .sort("count", descending=True)
        .head(10)
    )
    display(type_dist_inactive)

# Por exchange
print("\n🏛️ DISTRIBUCIÓN POR PRIMARY EXCHANGE:")
print("\n   ACTIVOS:")
exchange_dist_active = (
    df_active.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
)
display(exchange_dist_active)

if len(df_inactive) > 0:
    print("\n   INACTIVOS:")
    exchange_dist_inactive = (
        df_inactive.group_by("primary_exchange")
        .agg(pl.len().alias("count"))
        .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
        .sort("count", descending=True)
        .head(10)
    )
    display(exchange_dist_inactive)

# =============================================================================
# ESTADÍSTICAS DE CIK
# =============================================================================
print("\n📋 ESTADÍSTICAS DE CIK (SEC Identifier):")
print("\n   ACTIVOS:")
cik_active = df_active.filter(pl.col("cik").is_not_null())
print(f"      ✅ Con CIK: {len(cik_active):,} ({len(cik_active)/len(df_active)*100:.1f}%)")
print(f"      ❌ Sin CIK: {df_active['cik'].null_count():,} ({df_active['cik'].null_count()/len(df_active)*100:.1f}%)")

if len(df_inactive) > 0:
    print("\n   INACTIVOS:")
    cik_inactive = df_inactive.filter(pl.col("cik").is_not_null())
    print(f"      ✅ Con CIK: {len(cik_inactive):,} ({len(cik_inactive)/len(df_inactive)*100:.1f}%)")
    print(f"      ❌ Sin CIK: {df_inactive['cik'].null_count():,} ({df_inactive['cik'].null_count()/len(df_inactive)*100:.1f}%)")
print()

# =============================================================================
# ESTADÍSTICAS DE FIGI
# =============================================================================
print("📋 ESTADÍSTICAS DE FIGI (Bloomberg Identifier):")
print("\n   ACTIVOS:")
figi_active = df_active.filter(pl.col("composite_figi").is_not_null())
print(f"      ✅ Con FIGI: {len(figi_active):,} ({len(figi_active)/len(df_active)*100:.1f}%)")
print(f"      ❌ Sin FIGI: {df_active['composite_figi'].null_count():,} ({df_active['composite_figi'].null_count()/len(df_active)*100:.1f}%)")

if len(df_inactive) > 0:
    print("\n   INACTIVOS:")
    figi_inactive = df_inactive.filter(pl.col("composite_figi").is_not_null())
    print(f"      ✅ Con FIGI: {len(figi_inactive):,} ({len(figi_inactive)/len(df_inactive)*100:.1f}%)")
    print(f"      ❌ Sin FIGI: {df_inactive['composite_figi'].null_count():,} ({df_inactive['composite_figi'].null_count()/len(df_inactive)*100:.1f}%)")
print()

# =============================================================================
# RESUMEN EJECUTIVO - ACTIVOS
# =============================================================================
print("=" * 100)
print("RESUMEN EJECUTIVO - TICKERS ACTIVOS")
print("=" * 100)
print(f"""
 UNIVERSO ACTIVO: {len(df_active):,} tickers

 COMPOSICIÓN:
   • Common Stocks (CS):    {len(df_active.filter(pl.col('type') == 'CS')):>5,} ({len(df_active.filter(pl.col('type') == 'CS'))/len(df_active)*100:>5.1f}%)
   • ETFs:                  {len(df_active.filter(pl.col('type') == 'ETF')):>5,} ({len(df_active.filter(pl.col('type') == 'ETF'))/len(df_active)*100:>5.1f}%)
   • Preferred (PFD):       {len(df_active.filter(pl.col('type') == 'PFD')):>5,} ({len(df_active.filter(pl.col('type') == 'PFD'))/len(df_active)*100:>5.1f}%)
   • Warrants:              {len(df_active.filter(pl.col('type') == 'WARRANT')):>5,} ({len(df_active.filter(pl.col('type') == 'WARRANT'))/len(df_active)*100:>5.1f}%)
   • ADRs:                  {len(df_active.filter(pl.col('type') == 'ADRC')):>5,} ({len(df_active.filter(pl.col('type') == 'ADRC'))/len(df_active)*100:>5.1f}%)
   • Funds:                 {len(df_active.filter(pl.col('type') == 'FUND')):>5,} ({len(df_active.filter(pl.col('type') == 'FUND'))/len(df_active)*100:>5.1f}%)

 EXCHANGES:
   • Nasdaq (XNAS):         {len(df_active.filter(pl.col('primary_exchange') == 'XNAS')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'XNAS'))/len(df_active)*100:>5.1f}%)
   • NYSE (XNYS):           {len(df_active.filter(pl.col('primary_exchange') == 'XNYS')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'XNYS'))/len(df_active)*100:>5.1f}%)
   • NYSE Arca (ARCX):      {len(df_active.filter(pl.col('primary_exchange') == 'ARCX')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'ARCX'))/len(df_active)*100:>5.1f}%)
   • BATS:                  {len(df_active.filter(pl.col('primary_exchange') == 'BATS')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'BATS'))/len(df_active)*100:>5.1f}%)
   • NYSE American (XASE):  {len(df_active.filter(pl.col('primary_exchange') == 'XASE')):>5,} ({len(df_active.filter(pl.col('primary_exchange') == 'XASE'))/len(df_active)*100:>5.1f}%)

 CALIDAD DE DATOS:
   • {len(cik_active)/len(df_active)*100:.1f}% con CIK (SEC identifier)
   • {len(figi_active)/len(df_active)*100:.1f}% con FIGI (Bloomberg identifier)
""")

# =============================================================================
# RESUMEN EJECUTIVO - INACTIVOS
# =============================================================================
if len(df_inactive) > 0:
    print("=" * 100)
    print("RESUMEN EJECUTIVO - TICKERS INACTIVOS")
    print("=" * 100)
    
    cik_inactive_calc = df_inactive.filter(pl.col("cik").is_not_null())
    figi_inactive_calc = df_inactive.filter(pl.col("composite_figi").is_not_null())
    
    print(f"""
 UNIVERSO INACTIVO: {len(df_inactive):,} tickers (delisted/merged/inactive)

 COMPOSICIÓN:
   • Common Stocks (CS):    {len(df_inactive.filter(pl.col('type') == 'CS')):>5,} ({len(df_inactive.filter(pl.col('type') == 'CS'))/len(df_inactive)*100:>5.1f}%)
   • ETFs:                  {len(df_inactive.filter(pl.col('type') == 'ETF')):>5,} ({len(df_inactive.filter(pl.col('type') == 'ETF'))/len(df_inactive)*100:>5.1f}%)
   • Preferred (PFD):       {len(df_inactive.filter(pl.col('type') == 'PFD')):>5,} ({len(df_inactive.filter(pl.col('type') == 'PFD'))/len(df_inactive)*100:>5.1f}%)
   • Warrants:              {len(df_inactive.filter(pl.col('type') == 'WARRANT')):>5,} ({len(df_inactive.filter(pl.col('type') == 'WARRANT'))/len(df_inactive)*100:>5.1f}%)

 CALIDAD DE DATOS:
   • {len(cik_inactive_calc)/len(df_inactive)*100:.1f}% con CIK (SEC identifier)
   • {len(figi_inactive_calc)/len(df_inactive)*100:.1f}% con FIGI (Bloomberg identifier)
""")

print("=" * 100)
print("ANÁLISIS COMPLETADO")
print("=" * 100)

📂 Directorio de trabajo: D:\04_TRADING_SMALLCAPS

📥 Cargando datos desde: D:\04_TRADING_SMALLCAPS\raw\polygon\reference\tickers_snapshot\snapshot_date=2025-10-19\tickers.parquet
   Archivo existe: True

ANÁLISIS UNIVERSO COMPLETO - TICKERS ACTIVOS + INACTIVOS

TOTAL TICKERS (todos): 11,845
TOTAL COLUMNAS: 13

✅ TICKERS ACTIVOS:   11,845 (100.0%)
❌ TICKERS INACTIVOS:      0 (  0.0%)

COLUMNAS Y TIPOS DE DATOS
ticker                         | String               | Nulls:      0 (  0.0%)
name                           | String               | Nulls:      0 (  0.0%)
market                         | String               | Nulls:      0 (  0.0%)
locale                         | String               | Nulls:      0 (  0.0%)
primary_exchange               | String               | Nulls:      0 (  0.0%)
type                           | String               | Nulls:      0 (  0.0%)
active                         | Boolean              | Nulls:      0 (  0.0%)
currency_name                  | St

ticker,name,market,primary_exchange,type,active,cik
str,str,str,str,str,bool,str
"""CMCI""","""VanEck CMCI Commodity Strategy…","""stocks""","""BATS""","""ETF""",true,null
"""QBSF""","""AllianzIM U.S. Equity Buffer15…","""stocks""","""BATS""","""ETF""",true,"""0001797318"""
"""OCCIO""","""OFS Credit Company, Inc. 6.125…","""stocks""","""XNAS""","""PFD""",true,"""0001716951"""
"""SDHIR""","""Siddhi Acquisition Corp Right""","""stocks""","""XNAS""","""RIGHT""",true,"""0002034037"""
"""USCA""","""Xtrackers MSCI USA Climate Act…","""stocks""","""ARCX""","""ETF""",true,null
"""SWZ""","""Total Return Securities Fund""","""stocks""","""XNYS""","""FUND""",true,"""0000813623"""
"""VSDB""","""Vanguard Short Duration Bond E…","""stocks""","""BATS""","""ETF""",true,"""0000836906"""
"""JLS""","""Nuveen Mortgage and Income Fun…","""stocks""","""XNYS""","""FUND""",true,"""0001472215"""
"""JPC""","""Nuveen Preferred & Income Oppo…","""stocks""","""XNYS""","""FUND""",true,"""0001216583"""



TAIL(10) - ÚLTIMAS 10 ACCIONES


ticker,name,market,primary_exchange,type,active,cik
str,str,str,str,str,bool,str
"""ACSG""","""American Century Small Cap Gro…","""stocks""","""BATS""","""ETF""",true,null
"""AVSF""","""Avantis Short-Term Fixed Incom…","""stocks""","""ARCX""","""ETF""",true,"""0001481714"""
"""HVT.A""","""Haverty Furniture Companies, I…","""stocks""","""XNYS""","""CS""",true,"""0000216085"""
"""RNG""","""RINGCENTRAL, INC.""","""stocks""","""XNYS""","""CS""",true,"""0001384905"""
"""CORN""","""Teucrium Corn Fund""","""stocks""","""ARCX""","""ETV""",true,"""0001471824"""
"""MDV""","""Modiv Industrial, Inc.""","""stocks""","""XNYS""","""CS""",true,"""0001645873"""
"""RINT""","""Russell Investments Exchange T…","""stocks""","""XNAS""","""ETF""",true,"""0002042513"""
"""FNDX""","""Schwab Fundamental U.S. Large …","""stocks""","""ARCX""","""ETF""",true,"""0001454889"""
"""SZNE""","""Pacer CFRA- Stoval Equal Weigh…","""stocks""","""ARCX""","""ETF""",true,"""0001616668"""



COMPARACIÓN: ACTIVOS VS INACTIVOS

 DISTRIBUCIÓN POR TIPO DE ACTIVO (type):

   ACTIVOS:


type,count,percentage
str,u32,f64
"""CS""",5226,44.119882
"""ETF""",4361,36.817222
"""PFD""",441,3.72309
"""WARRANT""",418,3.528915
"""ADRC""",389,3.284086
"""FUND""",362,3.056142
"""UNIT""",174,1.468974
"""SP""",159,1.342339
"""ETS""",123,1.038413



🏛️ DISTRIBUCIÓN POR PRIMARY EXCHANGE:

   ACTIVOS:


primary_exchange,count,percentage
str,u32,f64
"""XNAS""",5127,43.284086
"""XNYS""",2882,24.330941
"""ARCX""",2473,20.878008
"""BATS""",1061,8.957366
"""XASE""",302,2.549599



📋 ESTADÍSTICAS DE CIK (SEC Identifier):

   ACTIVOS:
      ✅ Con CIK: 10,555 (89.1%)
      ❌ Sin CIK: 1,290 (10.9%)

📋 ESTADÍSTICAS DE FIGI (Bloomberg Identifier):

   ACTIVOS:
      ✅ Con FIGI: 9,840 (83.1%)
      ❌ Sin FIGI: 2,005 (16.9%)

RESUMEN EJECUTIVO - TICKERS ACTIVOS

 UNIVERSO ACTIVO: 11,845 tickers

 COMPOSICIÓN:
   • Common Stocks (CS):    5,226 ( 44.1%)
   • ETFs:                  4,361 ( 36.8%)
   • Preferred (PFD):         441 (  3.7%)
   • Warrants:                418 (  3.5%)
   • ADRs:                    389 (  3.3%)
   • Funds:                   362 (  3.1%)

 EXCHANGES:
   • Nasdaq (XNAS):         5,127 ( 43.3%)
   • NYSE (XNYS):           2,882 ( 24.3%)
   • NYSE Arca (ARCX):      2,473 ( 20.9%)
   • BATS:                  1,061 (  9.0%)
   • NYSE American (XASE):    302 (  2.5%)

 CALIDAD DE DATOS:
   • 89.1% con CIK (SEC identifier)
   • 83.1% con FIGI (Bloomberg identifier)

ANÁLISIS COMPLETADO


In [17]:
"""
Análisis Completo: Tickers Activos + Inactivos
Snapshot: 2025-10-24
"""
import polars as pl
from pathlib import Path
import os

# Configurar
project_root = Path(r"D:\04_TRADING_SMALLCAPS")
os.chdir(project_root)

# =============================================================================
# CARGAR RESUMEN CSV
# =============================================================================
csv_path = "temp_active_counts_complete.csv"
df_summary = pl.read_csv(csv_path)

print("="*100)
print("RESUMEN GENERAL")
print("="*100)
display(df_summary)
print()

# =============================================================================
# CARGAR SNAPSHOT COMPLETO
# =============================================================================
snapshot_path = "raw/polygon/reference/tickers_snapshot/snapshot_date=2025-10-24/tickers_all.parquet"
df_all = pl.read_parquet(snapshot_path)

print(f"TOTAL TICKERS: {len(df_all):,}")
print()

# Separar activos e inactivos
df_active = df_all.filter(pl.col("active") == True)
df_inactive = df_all.filter(pl.col("active") == False)

print(f"ACTIVOS:   {len(df_active):>8,} ({len(df_active)/len(df_all)*100:>5.1f}%)")
print(f"INACTIVOS: {len(df_inactive):>8,} ({len(df_inactive)/len(df_all)*100:>5.1f}%)")
print()

# =============================================================================
# HEAD(10) - ACTIVOS
# =============================================================================
print("="*100)
print("HEAD(10) - TICKERS ACTIVOS")
print("="*100)
cols = ["ticker", "name", "market", "primary_exchange", "type", "active", "cik"]
display(df_active.select(cols).head(10))

# =============================================================================
# HEAD(10) - INACTIVOS
# =============================================================================
print()
print("="*100)
print(f"HEAD(10) - TICKERS INACTIVOS (Total: {len(df_inactive):,})")
print("="*100)
display(df_inactive.select(cols).head(10))

# =============================================================================
# TAIL(10) - INACTIVOS  
# =============================================================================
print()
print("="*100)
print("TAIL(10) - TICKERS INACTIVOS")
print("="*100)
display(df_inactive.select(cols).tail(10))

# =============================================================================
# COMPARACIÓN POR TIPO
# =============================================================================
print()
print("="*100)
print("COMPARACIÓN: DISTRIBUCIÓN POR TIPO DE ACTIVO")
print("="*100)

print("\n📊 ACTIVOS:")
type_active = (
    df_active.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_active)

print("\n❌ INACTIVOS:")
type_inactive = (
    df_inactive.group_by("type")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(type_inactive)

# =============================================================================
# COMPARACIÓN POR EXCHANGE
# =============================================================================
print()
print("="*100)
print("COMPARACIÓN: DISTRIBUCIÓN POR EXCHANGE")
print("="*100)

print("\n📊 ACTIVOS:")
exchange_active = (
    df_active.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_active) * 100).alias("percentage"))
    .sort("count", descending=True)
)
display(exchange_active)

print("\n❌ INACTIVOS:")
exchange_inactive = (
    df_inactive.group_by("primary_exchange")
    .agg(pl.len().alias("count"))
    .with_columns((pl.col("count") / len(df_inactive) * 100).alias("percentage"))
    .sort("count", descending=True)
    .head(10)
)
display(exchange_inactive)

# =============================================================================
# RESUMEN EJECUTIVO
# =============================================================================
print()
print("="*100)
print("RESUMEN EJECUTIVO")
print("="*100)
print(f"""
📊 UNIVERSO TOTAL: {len(df_all):,} tickers

ACTIVOS ({len(df_active):,}):
   • Common Stocks (CS):  {len(df_active.filter(pl.col('type') == 'CS')):>6,}
   • ETFs:                {len(df_active.filter(pl.col('type') == 'ETF')):>6,}
   • Nasdaq (XNAS):       {len(df_active.filter(pl.col('primary_exchange') == 'XNAS')):>6,}
   • NYSE (XNYS):         {len(df_active.filter(pl.col('primary_exchange') == 'XNYS')):>6,}

INACTIVOS ({len(df_inactive):,}):
   • Common Stocks (CS):  {len(df_inactive.filter(pl.col('type') == 'CS')):>6,}
   • Warrants:            {len(df_inactive.filter(pl.col('type') == 'WARRANT')):>6,}
   • ETFs:                {len(df_inactive.filter(pl.col('type') == 'ETF')):>6,}
   • Delisted/Merged:     {len(df_inactive):,} (100% inactivos)
""")

print("="*100)
print("ANÁLISIS COMPLETADO")
print("="*100)

RESUMEN GENERAL


active,count,percentage
bool,i64,f64
true,11853,34.47644
false,22527,65.52356



TOTAL TICKERS: 34,380

ACTIVOS:     11,853 ( 34.5%)
INACTIVOS:   22,527 ( 65.5%)

HEAD(10) - TICKERS ACTIVOS


ticker,name,market,primary_exchange,type,active,cik
str,str,str,str,str,bool,str
"""A""","""Agilent Technologies Inc.""","""stocks""","""XNYS""","""CS""",true,"""0001090872"""
"""AA""","""Alcoa Corporation""","""stocks""","""XNYS""","""CS""",true,"""0001675149"""
"""AAA""","""Alternative Access First Prior…","""stocks""","""ARCX""","""ETF""",true,"""0001776878"""
"""AAAA""","""Amplius Aggressive Asset Alloc…","""stocks""","""BATS""","""ETF""",true,null
"""AAAU""","""Goldman Sachs Physical Gold ET…","""stocks""","""BATS""","""ETF""",true,"""0001708646"""
"""AACB""","""Artius II Acquisition Inc. Cla…","""stocks""","""XNAS""","""CS""",true,"""0002034334"""
"""AACBR""","""Artius II Acquisition Inc. Rig…","""stocks""","""XNAS""","""RIGHT""",true,"""0002034334"""
"""AACBU""","""Artius II Acquisition Inc. Uni…","""stocks""","""XNAS""","""UNIT""",true,"""0002034334"""
"""AACG""","""ATA Creativity Global American…","""stocks""","""XNAS""","""ADRC""",true,"""0001420529"""



HEAD(10) - TICKERS INACTIVOS (Total: 22,527)


...


COMPARACIÓN: DISTRIBUCIÓN POR EXCHANGE

📊 ACTIVOS:


primary_exchange,count,percentage
str,u32,f64
"""XNAS""",5133,43.305492
"""XNYS""",2880,24.297646
"""ARCX""",2475,20.88079
"""BATS""",1063,8.968194
"""XASE""",302,2.547878



❌ INACTIVOS:


primary_exchange,count,percentage
str,u32,f64
"""XNYS""",7926,35.184445
"""XNAS""",6599,29.293736
null,4166,18.493364
"""ARCX""",1824,8.09695
"""XASE""",1658,7.360057
"""BATS""",353,1.567008
"""XBOS""",1,0.004439



RESUMEN EJECUTIVO

📊 UNIVERSO TOTAL: 34,380 tickers

ACTIVOS (11,853):
   • Common Stocks (CS):   5,229
   • ETFs:                 4,365
   • Nasdaq (XNAS):        5,133
   • NYSE (XNYS):          2,880

INACTIVOS (22,527):
   • Common Stocks (CS):   6,242
   • Warrants:             1,724
   • ETFs:                 1,363
   • Delisted/Merged:     22,527 (100% inactivos)

ANÁLISIS COMPLETADO
